In [1]:
import warnings
warnings.filterwarnings("ignore")
# import ST_utils
# import train_STAligner
import STAligner

# the location of R (used for the mclust clustering)
import os
os.environ['R_HOME'] = "/home/tguo/tguo2/miniconda3/envs/env_STAligner/lib/R"
os.environ['R_USER'] = "/home/tguo/tguo2/miniconda3/envs/env_STAligner/lib/python3.7/site-packages/rpy2"
import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri


import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.sparse as sp
import scipy.linalg
from scipy import sparse

import torch
used_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
dirs="/data02/tguo/space_batch_effect/simulate/"
batch_sim="_1"
types=""
section_ids = ['1','2']
Batch_list = []
adj_list = []
print(section_ids)
flags=batch_sim+types
for section_id in section_ids:
    print(section_id)
    count=pd.read_csv(dirs+"gtt_input/feat"+section_id+flags+".csv",header=0,index_col=0,sep=',')
    adata=sc.AnnData(count)
    adata.X=sparse.csr_matrix(adata.X)
    adata.var_names_make_unique(join="++")

    # read the annotation
    meta = pd.read_csv(dirs+"gtt_input/meta"+section_id+flags+".csv", sep=',', header=0, index_col=0)
    
    adata.obs['celltype'] = meta.loc[adata.obs_names, 'celltype']
    adata.obs['batch'] = meta.loc[adata.obs_names, 'batch']
    
    coord=pd.read_csv(dirs+"gtt_input/coord"+section_id+flags+".csv",header=0,index_col=0,sep=',')
    coord=coord.loc[:,['x','y']]
    adata.obsm["spatial"] = coord.to_numpy()
    # Constructing the spatial network
    STAligner.Cal_Spatial_Net(adata, k_cutoff=10,model='KNN') # the spatial network are saved in adata.uns[‘adj’]
    # STAligner.Stats_Spatial_Net(adata) # plot the number of spatial neighbors

    adj_list.append(adata.uns['adj'])
    Batch_list.append(adata)

['1', '2']
1
------Calculating spatial graph...
The graph contains 5000 edges, 500 cells.
10.0000 neighbors per cell on average.
2
------Calculating spatial graph...
The graph contains 5000 edges, 500 cells.
10.0000 neighbors per cell on average.


In [3]:
adata_concat = ad.concat(Batch_list, label="slice_name", keys=section_ids)
adata_concat.obs['celltype'] = adata_concat.obs['celltype']
adata_concat.obs["batch_name"] = adata_concat.obs["batch"].astype('category')
print('adata_concat.shape: ', adata_concat.shape)
adj_concat = np.asarray(adj_list[0].todense())
for batch_id in range(1,len(section_ids)):
    adj_concat = scipy.linalg.block_diag(adj_concat, np.asarray(adj_list[batch_id].todense()))
adata_concat.uns['edgeList'] = np.nonzero(adj_concat)

adata_concat.shape:  (1000, 500)


In [4]:
%%time
adata_concat = STAligner.train_STAligner(adata_concat, verbose=True, knn_neigh = 100, device=used_device)
embed=adata_concat.obsm['STAligner']
embed_names=['STAligner-'+str(i) for i in np.arange(embed.shape[1])]
embed=pd.DataFrame(embed,index=adata_concat.obs_names,columns=embed_names)
embed.to_csv(dirs+"methods/STAligner_mat"+flags+'.txt')

STAligner(
  (conv1): GATConv(500, 512, heads=1)
  (conv2): GATConv(512, 30, heads=1)
  (conv3): GATConv(30, 512, heads=1)
  (conv4): GATConv(512, 500, heads=1)
)
Pretrain with STAGATE...


100%|██████████| 500/500 [00:05<00:00, 93.74it/s] 


Train with STAligner...


  0%|          | 0/500 [00:00<?, ?it/s]

Update spot triplets at epoch 500


...

Update spot triplets at epoch 800


 79%|███████▉  | 397/500 [00:06<00:01, 93.71it/s]

Update spot triplets at epoch 900


100%|██████████| 500/500 [00:07<00:00, 63.95it/s]

CPU times: user 16.7 s, sys: 1.5 s, total: 18.2 s
Wall time: 18.3 s


In [ ]:
celltype=adata_concat.obs['celltype'].values
batch=adata_concat.obs['batch_name'].values
ul=np.unique(celltype)
ub=np.unique(batch)
import umap.umap_ as umap
import matplotlib.pyplot as plt
umap_mat=umap.UMAP().fit_transform(adata_concat.obsm['STAligner'])
# x=PCA(n_components=30).fit_transform(SPII.feat.values)
# umap_mat=umap.UMAP().fit_transform(x)
colour1=['lightcoral','red','tomato','chocolate','darkorange','gold','olive','yellow','yellowgreen','lawngreen','forestgreen','lime',
       'cyan','deepskyblue','dodgerblue','royalblue','blue','blueviolet','purple','fuchsia','hotpink','pink','gray','black','teal']

colour2=['r','g','black','cyan','blue','purple','yellow','grey']
f, axs= plt.subplots(1,2,figsize=(20,10))
size=10
for i in range(len(ul)):
    axs[0].scatter(umap_mat[np.where(celltype==ul[i])[0],0],umap_mat[np.where(celltype==ul[i])[0],1],c=colour1[i],s=size)

axs[0].set_xlabel("umap1",fontsize=30)
axs[0].set_ylabel("umap2",fontsize=30)
axs[0].legend(ul,loc="best",fontsize=20,markerscale=4,bbox_to_anchor=(-0.19,1))
axs[0].tick_params(axis='both', which='major', labelsize=20)
for i in range(len(ub)):
    axs[1].scatter(umap_mat[np.where(batch==ub[i])[0],0],umap_mat[np.where(batch==ub[i])[0],1],c=colour2[i],s=size)
axs[1].set_xlabel("umap1",fontsize=30)
axs[1].legend(ub,loc="best",fontsize=20,markerscale=4,bbox_to_anchor=(1.4,1))
axs[1].tick_params(axis='both', which='major', labelsize=20)